# Holistic Regression

In [ ]:
# Loading required libraries
using JuMP, Gurobi, CSV, Plots

In [ ]:
# Importing data and splitting into training, validation and test sets (50%/25%/25%)
df = convert(Array, CSV.read("data1.csv"))
# df = convert(Array, CSV.read("data2.csv"))
Y = df[:,end]
X = df[:,1:end-1]
n, p_orig = size(X)
val_start = round(Int, 0.5 * n) 
test_start = round(Int, 0.75 * n) 
Y_train = Y[1:val_start-1,]
Y_val = Y[val_start:test_start-1,]
Y_test = Y[test_start:end,]
X_train_orig = X[1:val_start-1, :]
X_val_orig = X[val_start:test_start-1, :]
X_test_orig = X[test_start:end, :];

In [ ]:
# Applying transformations to the X variables
n, p_orig = size(X_train_orig)
transforms = [x -> x, x -> x.^2, x -> sqrt.(x), x -> log.(x)]
apply_transforms(X) = hcat([transforms[i](X) for i = 1:length(transforms)]...)
p = p_orig * length(transforms)
X_train = apply_transforms(X_train_orig)
X_val = apply_transforms(X_val_orig)
X_test = apply_transforms(X_test_orig);

In [ ]:
# Normalising the data
mean_X_train = mean(X_train,1)
denom_X_train = std(X_train,1)
X_train = (X_train .- mean_X_train) ./ denom_X_train
X_val = (X_val .- mean_X_train) ./ denom_X_train
X_test = (X_test .- mean_X_train) ./ denom_X_train
mean_Y_train = mean(Y_train);
Y_train = Y_train .- mean_Y_train;
Y_val = Y_val .- mean_Y_train
Y_test = Y_test .- mean_Y_train;

In [ ]:
# Finding pairwise correlations
function sorted_correlations(cor_matrix)
  c = copy(abs.(cor_matrix))
  p = length(cor_matrix[1,:])
  num_pairs = convert(Int64,p*(p-1)/2)
  pair_list = zeros(Int64, num_pairs, 2)
  magnitude = zeros(num_pairs)

  # Set lower triangular correlation values = 0
  for i=1:p
    for j=1:i
      c[i,j] = 0
    end
  end

  for i=1:num_pairs
    ind = indmax(c)
    col = floor(ind/p) + 1
    row = ind % p
    magnitude[i] = c[ind]
    pair_list[i,1] = row
    pair_list[i,2] = col
    c[ind] = 0
  end

  return(pair_list, magnitude)
end

cor_matrix = cor(X_train[:,1:p_orig])
pair_list, magnitude = sorted_correlations(cor_matrix)
num_pairs = length(magnitude)
pair_list;

In [ ]:
# Mixed-Integer Optimization
Big_M = fill(4, p)
K = 5 # sparsity parameter
Γ = 0.1 # robustness parameter 
ρ = 0.8 # pairwise multicollinearity threshold 
num_opt = length(transforms)
print_model = false
OutputFlag = 0
robustness = true

m = Model(solver = GurobiSolver(OutputFlag=OutputFlag))
    @variable(m, β[1:p])
    @variable(m, z[1:p], Bin)
    @variable(m, t >= 0)
    @variable(m, θ >= 0)

    @objective(m, Min, t + Γ * θ)
    @constraint(m, norm(Y_train - X_train * β) <= t)

    # Sparsity constraints
    @constraint(m, m_gt[d=1:p], β[d] <= Big_M[d] * z[d])
    @constraint(m, m_lt[d=1:p], -Big_M[d] * z[d] <= β[d])
    @constraint(m, sparsity, sum(z[d] for d = 1:p) <= K)

    # Robustness constraint
    if robustness
      @variable(m, β_0[1:p] >= 0)
      @constraint(m, beta_pos[j=1:p], β_0[j] >= β[j])
      @constraint(m, beta_neg[j=1:p], β_0[j] >= -β[j])
      @constraint(m, sum(β_0) <= θ)
    end

    # Pairwise multicolinearity constraints 

    for i=1:num_pairs
      if magnitude[i] > ρ
        ind1 = pair_list[i,1]
        ind2 = pair_list[i,2]
        @constraint(m, z[ind1] + z[ind2] + z[ind1 + p_orig] + z[ind2 + p_orig] 
            + z[ind1 + 2*p_orig] + z[ind2 + 2*p_orig] + z[ind1 + 3*p_orig] + z[ind2 + 3*p_orig] <= 1)
      else
        break
      end
    end

    # Transformation constraints
    if num_opt > 1
      @constraint(m, non_linear[j=1:p_orig], sum(z[j + t*p_orig] for t = 0:(num_opt-1)) <= 1)
    end

if print_model
  print(m)
end

In [ ]:
solve(m)

In [ ]:
# Finding the optimal model
status = solve(m)
z_soln = getvalue(z)
β_soln = getvalue(β)
println("Status = ", status)
println("Selected variables = ", find(z_soln))
println("Regression Equation:")
println("y = ", join(["$(round(β_soln[i],3)) x[$i]" for i in find(z_soln)], " + "))

In [ ]:
# Performance of optimal model
Y_hat_train = X_train*β_soln
R2_train = 1 - sum((Y_hat_train - Y_train).^2) / sum((mean(Y_train) - Y_train).^2)
Y_hat_test = X_test*β_soln
R2_test = 1 - sum((Y_hat_test - Y_test).^2) / sum((mean(Y_train) - Y_test).^2)
z_soln_manual = abs.(β_soln) .> 0.0000001
max_corr = maximum(abs.(cor_matrix[z_soln_manual,z_soln_manual]) - eye(K))

println("***PARAMETERS***")
println("Sparsity parameter K = ", K)
println("Pairwise correlation threshold ρ = ", ρ)
println("Robustness parameter Γ = ", Γ)
println()
println("***RESULTS***")
println("max corr:\t", max_corr)
println("MIO R2 train\t", R2_train)
println("MIO R2 test\t", R2_test)

In [ ]:
# Tuning
for K = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
# for K = [1, 2, 3, 4, 5, 6, 7, 8]
    for Γ = [0.01, 0.1, 0.5, 1.0]
        m = Model(solver = GurobiSolver(OutputFlag=OutputFlag))
                
                best = zeros(0)
        
                @variable(m, β[1:p])
                @variable(m, z[1:p], Bin)
                @variable(m, t >= 0)
                @variable(m, θ >= 0)

                @objective(m, Min, t + Γ * θ)
                @constraint(m, norm(Y_train - X_train * β) <= t)

                # Sparsity constraints
                @constraint(m, m_gt[d=1:p], β[d] <= Big_M[d] * z[d])
                @constraint(m, m_lt[d=1:p], -Big_M[d] * z[d] <= β[d])
                @constraint(m, sparsity, sum(z[d] for d = 1:p) <= K)

                # Robustness constraint
                if robustness
                  @variable(m, β_0[1:p] >= 0)
                  @constraint(m, beta_pos[j=1:p], β_0[j] >= β[j])
                  @constraint(m, beta_neg[j=1:p], β_0[j] >= -β[j])
                  @constraint(m, sum(β_0) <= θ)
                end

                # Pairwise multicolinearity constraints
                for i=1:num_pairs
                  if magnitude[i] > ρ
                    @constraint(m, z[pair_list[i,1]] + z[pair_list[i,2]] <= 1)
                  else
                    break
                  end
                end

                # Transformation constraints
                if num_opt > 1
                  @constraint(m, non_linear[j=1:p_orig], sum(z[j + t*p_orig] for t = 0:(num_opt-1)) <= 1)
                end

        status = solve(m)

        z_soln = getvalue(z)
        β_soln = getvalue(β)
        println()
        println("Status = ", status)
        println("Selected variables = ", find(z_soln))
        println("Regression Equation:")
        println("y = ", join(["$(round(β_soln[i],3)) x[$i]" for i in find(z_soln)], " + "))

        Y_hat_val = X_val*β_soln
        R2_val = 1 - sum((Y_hat_val - Y_val).^2) / sum((mean(Y_train) - Y_val).^2)
        push!(best, R2_val)
        
        println("***PARAMETERS***")
        println("Sparsity parameter K = ", K)
        println("Pairwise correlation threshold ρ = ", ρ)
        println("Robustness parameter Γ = ", Γ)
        println()
        println("***RESULTS***")
        println("MIO R2 val\t", R2_val)
        println("************************")
        
    end
end

In [ ]:
# Optimal Values
X_train = [X_train;X_val]
Y_train = [Y_train;Y_val]

# K = 4      # for data1.csv
# Γ = 0.1    # for data1.csv

K = 4        # for data2.csv
Γ = 0.01     # for data2.csv

m = Model(solver = GurobiSolver(OutputFlag=OutputFlag))
    @variable(m, β[1:p])
    @variable(m, z[1:p], Bin)
    @variable(m, t >= 0)
    @variable(m, θ >= 0)

    @objective(m, Min, t + Γ * θ)
    @constraint(m, norm(Y_train - X_train * β) <= t)

    # Sparsity constraints
    @constraint(m, m_gt[d=1:p], β[d] <= Big_M[d] * z[d])
    @constraint(m, m_lt[d=1:p], -Big_M[d] * z[d] <= β[d])
    @constraint(m, sparsity, sum(z[d] for d = 1:p) <= K)

    # Robustness constraint
    if robustness
        @variable(m, β_0[1:p] >= 0)
        @constraint(m, beta_pos[j=1:p], β_0[j] >= β[j])
        @constraint(m, beta_neg[j=1:p], β_0[j] >= -β[j])
        @constraint(m, sum(β_0) <= θ)
    end

    # Pairwise multicolinearity constraints
    for i=1:num_pairs
        if magnitude[i] > ρ
            @constraint(m, z[pair_list[i,1]] + z[pair_list[i,2]] <= 1)
        else
            break
        end
    end
    
    # Transformation constraints
    if num_opt > 1
        @constraint(m, non_linear[j=1:p_orig], sum(z[j + t*p_orig] for t = 0:(num_opt-1)) <= 1)
    end

status = solve(m)

z_soln = getvalue(z)
β_soln = getvalue(β)
println()
println("Status = ", status)
println("Selected variables = ", find(z_soln))
println("Regression Equation:")
println("y = ", join(["$(round(β_soln[i],3)) x[$i]" for i in find(z_soln)], " + "))

Y_hat_train = X_train*β_soln
Y_hat_test = X_test*β_soln
R2_train = 1 - sum((Y_hat_train - Y_train).^2) / sum((mean(Y_train) - Y_train).^2)
R2_test = 1 - sum((Y_hat_test - Y_test).^2) / sum((mean(Y_train) - Y_test).^2)
        
println("***PARAMETERS***")
println("Sparsity parameter K = ", K)
println("Pairwise correlation threshold ρ = ", ρ)
println("Robustness parameter Γ = ", Γ)
println()
println("***RESULTS***")
println("MIO R2 train\t", R2_train)
println("MIO R2 test\t", R2_test)
println("************************")
    
    
    
    